In [284]:
import pandas as pd
import numpy as np

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objs as go

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [285]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [286]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [287]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [288]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

QUESTION1:  I tried to use queries first and then concatenate final dataset , then proceeded to create a dash app, however, i was not able to figure out using 2 input drop downs on selection. I received an error trying to use 2 drop downs for interactive selection. Hence, for filtered_data I used just one drop down to use for selection for spc_Common to show the bar graphs. 

In [289]:
##Get query for each boro
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,spc_common,health,status,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=boroname,spc_common,health,status').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees
soql_trees['count_tree_id'].sum()

len(soql_trees)





355

In [290]:
trees['boroname'].unique()

array(['Queens', 'Brooklyn', 'Manhattan', 'Staten Island', 'Bronx'],
      dtype=object)

In [291]:
#drop NAN

soql_trees[soql_trees['spc_common'].isna()].sum()




boroname         BronxBronxBronx
spc_common                     0
status            AliveDeadStump
count_tree_id               4619
dtype: object

In [292]:

df1 = soql_trees.dropna(axis=0, how='any')
df1.sum()

# #count of total bronx trees
# df['count_tree_id'].sum()

boroname         BronxBronxBronxBronxBronxBronxBronxBronxBronxB...
spc_common       American beechAmerican beechAmerican beechAmer...
health           FairGoodPoorFairGoodPoorFairGoodPoorFairGoodPo...
status           AliveAliveAliveAliveAliveAliveAliveAliveAliveA...
count_tree_id                                                80584
dtype: object

In [293]:
df1_tot = df1.groupby(['boroname', 'spc_common'])['count_tree_id'].sum()
df1.head(10)




,boroname,spc_common,health,status,count_tree_id
0,Bronx,American beech,Fair,Alive,7
1,Bronx,American beech,Good,Alive,21
2,Bronx,American beech,Poor,Alive,3
3,Bronx,American elm,Fair,Alive,248
4,Bronx,American elm,Good,Alive,1176
5,Bronx,American elm,Poor,Alive,47
6,Bronx,American hophornbeam,Fair,Alive,28
7,Bronx,American hophornbeam,Good,Alive,148
8,Bronx,American hophornbeam,Poor,Alive,9
9,Bronx,American hornbeam,Fair,Alive,61


In [294]:
df1_totspecies = df1.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()
df1_totspecies


boroname  spc_common              health
Bronx     'Schubert' chokecherry  Fair       90
                                  Good      456
                                  Poor       29
          American beech          Fair        7
                                  Good       21
                                           ... 
          white pine              Fair        3
                                  Good       14
          willow oak              Fair       42
                                  Good      453
                                  Poor       10
Name: count_tree_id, Length: 352, dtype: int64

In [295]:
##Get query for each boro

soql_url1 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,health,status,boroname,count(tree_id)' +\
        '&$where=boroname=\'Brooklyn\'' +\
        '&$group=spc_common,health,status,boroname').replace(' ', '%20')
soql_trees_BK = pd.read_json(soql_url1)

soql_trees_BK['count_tree_id'].sum()
soql_trees_BK[soql_trees_BK['spc_common'].isna()].sum()

df2 = soql_trees_BK.dropna(axis=0, how='any')
df2.sum()
df2

df2_totspecies = df2.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()
df2_totspecies



# soql_trees_BK
# soql_trees_BK1 = soql_trees_BK['count_tree_id'].sum()
# soql_trees_BK1

len(soql_trees_BK)

##Get query for each boro

378

In [296]:
##Get query for each boro
soql_url2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,health,status,boroname,count(tree_id)' +\
        '&$where=boroname=\'Manhattan\'' +\
        '&$group=spc_common,health,status,boroname').replace(' ', '%20')
soql_trees_MN = pd.read_json(soql_url2)

soql_trees_MN['count_tree_id'].sum()
soql_trees_MN[soql_trees_MN['spc_common'].isna()].sum()

df3 = soql_trees_MN.dropna(axis=0, how='any')
df3.sum()
df3

df3_totspecies = df3.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()
df3_totspecies




boroname   spc_common              health
Manhattan  'Schubert' chokecherry  Fair       40
                                   Good      111
                                   Poor       12
           American beech          Fair        4
                                   Good       15
                                            ... 
           white oak               Poor       23
           white pine              Fair        1
           willow oak              Fair      115
                                   Good      747
                                   Poor       27
Name: count_tree_id, Length: 332, dtype: int64

In [297]:
##Get query for each boro
soql_url3 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,health,status,boroname,count(tree_id)' +\
        '&$where=boroname in (\'Queens\',\'Staten Island\')' +\
        '&$group=spc_common,health,status,boroname').replace(' ', '%20')
soql_trees_QST = pd.read_json(soql_url3)

soql_trees_QST['count_tree_id'].sum()
soql_trees_QST[soql_trees_QST['spc_common'].isna()].sum()

df4 = soql_trees_QST.dropna(axis=0, how='any')
df4.sum()
df4

df4_totspecies = df4.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()
df4_totspecies





boroname       spc_common              health
Queens         'Schubert' chokecherry  Fair       290
                                       Good      1650
                                       Poor        73
               American beech          Fair         8
                                       Good        73
                                                 ... 
Staten Island  white pine              Good        42
                                       Poor         1
               willow oak              Fair        37
                                       Good       210
                                       Poor        11
Name: count_tree_id, Length: 748, dtype: int64

In [298]:
##Concat to get final dataset

frames = [df1_totspecies, df2_totspecies, df3_totspecies,df4_totspecies]
final = pd.concat(frames)
final = final.to_frame()

final

#Calculate totals
fin_totals = final.groupby(['boroname', 'spc_common'])['count_tree_id'].sum()
fin_totals_spehealth= final.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()

fin_totals = fin_totals.reset_index(drop=False)
fin_totals_spehealth = fin_totals_spehealth.reset_index(drop=False)

# Merge to get final table and proportions
mergedata = pd.merge(fin_totals_spehealth, fin_totals, on=['boroname', 'spc_common'])

mergedata = mergedata.rename(columns={"count_tree_id_x": "tothealth", "count_tree_id_y": "totalspecies"})
# mergedata.columns.values = ['borocode', 'spc_common', 'tothealth', 'totalspecies']

mergedata['proportions'] = mergedata['tothealth']/mergedata['totalspecies']
mergedata 



,boroname,spc_common,health,tothealth,totalspecies,proportions
0,Bronx,'Schubert' chokecherry,Fair,90,575,0.156522
1,Bronx,'Schubert' chokecherry,Good,456,575,0.793043
2,Bronx,'Schubert' chokecherry,Poor,29,575,0.050435
3,Bronx,American beech,Fair,7,31,0.225806
4,Bronx,American beech,Good,21,31,0.677419
...,...,...,...,...,...,...
1803,Staten Island,white pine,Good,42,47,0.893617
1804,Staten Island,white pine,Poor,1,47,0.021277
1805,Staten Island,willow oak,Fair,37,258,0.143411
1806,Staten Island,willow oak,Good,210,258,0.813953


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [299]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

In [300]:
##user jupyterdash
app = JupyterDash(__name__)

In [301]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="Proportions review", className="header-title"
                ),
                html.P(
                    children="Analyze proportions",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Borough", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                               {"label": boroname, "value": boroname}
                        for boroname in np.sort(mergedata.boroname.unique())
                            ],
                            value="Bronx",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Type", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                 {"label": spc_common, "value": spc_common}
                        for spc_common in np.sort(mergedata.spc_common.unique())
                            ],
                            value="'Schubert' chokecherry",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),

            ],
            className="menu",
        ),
       html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="prop-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                
            ],
            className="wrapper",
        ),
    ]
)


@app.callback(
    Output('prop-chart', 'figure'),
    [Input("type-filter", "value")])
def update_figure(spc_common):

    filtered_data  = mergedata[mergedata.spc_common == spc_common]
   
    manhattan =  filtered_data [filtered_data.boroname == 'Manhattan']
    bronx =  filtered_data [filtered_data.boroname  == 'Bronx']
    brooklyn =  filtered_data [filtered_data.boroname  == 'Brooklyn']
    queens =  filtered_data [filtered_data.boroname == 'Queens']
    staten_island =  filtered_data [filtered_data.boroname == 'Staten Island']
    
    traces = []

    traces.append(go.Bar(
    x=queens['health'],
    y=queens['proportions'],
    name='Queens',
    opacity=0.9
    ))

    traces.append(go.Bar(
    x=manhattan['health'],
    y=manhattan['proportions'],
    name='Manhattan',
    opacity=0.9
    ))

    traces.append(go.Bar(
    x=bronx['health'],
    y=bronx['proportions'],
    name='Bronx',
    opacity=0.9
    ))

    traces.append(go.Bar(
    x=brooklyn['health'],
    y=brooklyn['proportions'],
    name='Brooklyn',
    opacity=0.9
    ))

    traces.append(go.Bar(
    x=staten_island['health'],
    y=staten_island['proportions'],
    name='Staten Island',
    opacity=0.9
    ))
    
    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Tree Analysis'},
            yaxis={'title': 'Proportion of trees'},
            margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
            legend=dict(x=-.1, y=1.2)
        )
    }


if __name__ == "__main__":
    app.run_server(debug=True,mode='inline')



QUESTION2: Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees?